In [3]:
import pandas as pd
import quandl

In [4]:
#get my quandl api key
import yaml
with open('/users/kevin/Documents/databases.yaml', 'r') as f:
        dbparams = yaml.load(f)

apikey = dbparams['Quandl']['apikey']

quandl.ApiConfig.api_key = apikey

In [11]:
# 2018 data for Cisco
data = quandl.get("WIKI/CSCO", start_date = '2018-01-01')

In [13]:
data


,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,,,,,,,,
2018-01-02,38.670,38.9500,38.4300,38.86,19972431.0,0.0,1.0,38.670,38.9500,38.4300,38.86,19972431.0
2018-01-03,38.720,39.2850,38.5300,39.17,29066090.0,0.0,1.0,38.720,39.2850,38.5300,39.17,29066090.0
2018-01-04,39.050,39.5400,38.9300,38.99,20606344.0,0.0,1.0,39.050,39.5400,38.9300,38.99,20606344.0
2018-01-05,39.550,39.8800,39.3650,39.53,24369510.0,0.0,1.0,39.550,39.8800,39.3650,39.53,24369510.0
2018-01-08,39.520,39.9600,39.3500,39.94,16511704.0,0.0,1.0,39.520,39.9600,39.3500,39.94,16511704.0
2018-01-09,39.790,39.9600,39.5400,39.69,21339760.0,0.0,1.0,39.790,39.9600,39.5400,39.69,21339760.0
2018-01-10,39.650,40.2400,39.6300,39.91,19110146.0,0.0,1.0,39.650,40.2400,39.6300,39.91,19110146.0
2018-01-11,40.140,40.2100,39.7500,40.10,20178596.0,0.0,1.0,40.140,40.2100,39.7500,40.10,20178596.0
2018-01-12,40.220,40.9300,40.0500,40.87,22962700.0,0.0,1.0,40.220,40.9300,40.0500,40.87,22962700.0


In [14]:
#list of dow companies from https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average 
dow_companies = ['MMM','AXP', 'AAPL','BA','CAT',
                 'CVX','CSCO','KO','DWDP','XOM',
                 'GS','HD','IBM','INTC','JNJ','JPM',
                 'MCD','MRK','MSFT','NKE','PFE',
                 'PG','TRV','UNH','UTX','VZ','V',
                 'WMT','WBA','DIS']

In [15]:
len(dow_companies)

30

In [16]:
#get the data for the first 3 dow companies:

cols = ['Open','High','Low','Close','Volume']
dow_data = {x:quandl.get('WIKI/'+x, start_date = '2018-01-01')[cols] for x in dow_companies[0:3]}

In [17]:
dow_data.keys()


dict_keys(['MMM', 'AXP', 'AAPL'])

In [18]:
dow_data['AAPL']

,Open,High,Low,Close,Volume
Date,,,,,
2018-01-02,170.160,172.3000,169.2600,172.260,25048048.0
2018-01-03,172.530,174.5500,171.9600,172.230,28819653.0
2018-01-04,172.540,173.4700,172.0800,173.030,22211345.0
2018-01-05,173.440,175.3700,173.0500,175.000,23016177.0
2018-01-08,174.350,175.6100,173.9300,174.350,20134092.0
2018-01-09,174.550,175.0600,173.4100,174.330,21262614.0
2018-01-10,173.160,174.3000,173.0000,174.290,23589129.0
2018-01-11,174.590,175.4886,174.4900,175.280,17523256.0
2018-01-12,176.180,177.3600,175.6500,177.090,25039531.0


In [19]:
#9-day trailing moving average
dow_data['AAPL'].rolling(window= 9, center=False).mean()

,Open,High,Low,Close,Volume
Date,,,,,
2018-01-02,NaN,NaN,NaN,NaN,NaN
2018-01-03,NaN,NaN,NaN,NaN,NaN
2018-01-04,NaN,NaN,NaN,NaN,NaN
2018-01-05,NaN,NaN,NaN,NaN,NaN
2018-01-08,NaN,NaN,NaN,NaN,NaN
2018-01-09,NaN,NaN,NaN,NaN,NaN
2018-01-10,NaN,NaN,NaN,NaN,NaN
2018-01-11,NaN,NaN,NaN,NaN,NaN
2018-01-12,173.500000,174.834289,172.981111,174.206667,2.296043e+07
